In [16]:
####加载包
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

In [29]:
###导入数据
data1 = pd.read_csv("data.csv",index_col=0,encoding="gbk")
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4754 entries, 5 to 11992
Data columns (total 89 columns):
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility                             4752 non-null float64
student_featur

In [30]:
print(data1.head(5))

     custid                          trade_no bank_card_no  \
5   2791858  20180507115231274000000023057383          卡号1   
10   534047  20180507121002192000000023073000          卡号1   
12  2849787  20180507125159718000000023114911          卡号1   
13  1809708  20180507121358683000000388283484          卡号1   
14  2499829  20180507115448545000000388205844          卡号1   

    low_volume_percent  middle_volume_percent  \
5                 0.01                   0.99   
10                0.02                   0.94   
12                0.04                   0.96   
13                0.00                   0.96   
14                0.01                   0.99   

    take_amount_in_later_12_month_highest  trans_amount_increase_rate_lately  \
5                                       0                               0.90   
10                                   2000                               1.28   
12                                      0                               1.00   
13          

In [31]:
#统计每列的缺失值个数
data1.isnull().sum().sort_values(ascending=False)

student_feature                               2998
cross_consume_count_last_1_month               426
query_org_count                                304
query_cash_count                               304
latest_six_month_apply                         304
latest_three_month_apply                       304
latest_query_time                              304
query_sum_count                                304
latest_one_month_apply                         304
query_finance_count                            304
apply_credibility                              304
apply_score                                    304
latest_query_day                               304
latest_one_month_loan                          297
loans_score                                    297
loans_credibility_behavior                     297
loans_count                                    297
loans_settle_count                             297
loans_overdue_count                            297
loans_org_count_behavior       

In [33]:
 #选择出为object类型的数据
obj_vars = []
def find_object(df):
    print(' object类型的变量 ')
    for col in df.columns:
        if df[col].dtype == 'object':
#             print(col)
            obj_vars.append(col)
    return obj_vars

In [34]:
find_object(data1)

 object类型的变量 


['trade_no',
 'bank_card_no',
 'reg_preference_for_trad',
 'source',
 'id_name',
 'latest_query_time',
 'loans_latest_time']

In [35]:
##分别计算每个特征缺失值情况，唯一值数量，唯一率等信息
def desc(df):
    loss_num=[] #缺失值的数量
    valid_num=[] #非缺失值的数量
    unique_num=[] #唯一的数量
    unique_percent=[] #唯一率
    for i,j in enumerate (df.columns): # enumerator 函数可以同时获得索引和值。 i表示索引，j表示值
        loss_num.append(df[df[j].isnull()].shape[0]) #isnull() 表示的是空 返回的是True（空）
        valid_num.append(df[df[j].notnull()].shape[0])
        unique_num.append(len(df[j].unique()))
        unique_percent.append(len(df[df[j].notnull()][j].unique())/df[df[j].notnull()].shape[0])
    data_desc = pd.DataFrame({"loss_num":loss_num,"valid_num":valid_num,"unique_num":unique_num,"unique_percent":unique_percent},index=df.columns)
    return data_desc
data_count = desc(data1)
data_count

,loss_num,unique_num,unique_percent,valid_num
custid,0,4754,1.000000,4754
trade_no,0,4754,1.000000,4754
bank_card_no,0,1,0.000210,4754
low_volume_percent,2,41,0.008418,4752
middle_volume_percent,2,91,0.018939,4752
take_amount_in_later_12_month_highest,0,166,0.034918,4754
trans_amount_increase_rate_lately,3,783,0.164597,4751
trans_activity_month,2,85,0.017677,4752
trans_activity_day,2,513,0.107744,4752
transd_mcc,2,42,0.008628,4752


In [45]:
data1[data1['trans_activity_month'].notnull()]

In [46]:
data1.dtypes.value_counts()

float64    70
int64      12
object      7
dtype: int64

In [47]:
#选择列
num_features = data1.select_dtypes(include = ["float64","int64"]).copy()
obj_features = data1.select_dtypes(include=['object']).copy()
obj_features_desc = desc(obj_features)
obj_features_desc

,loss_num,unique_num,unique_percent,valid_num
trade_no,0,4754,1.000000,4754
bank_card_no,0,1,0.000210,4754
reg_preference_for_trad,2,6,0.001052,4752
source,0,1,0.000210,4754
id_name,276,4310,0.962260,4478
latest_query_time,304,208,0.046517,4450
loans_latest_time,297,233,0.052053,4457


In [48]:
num_features

,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
5,2791858,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
10,534047,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
12,2849787,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
13,1809708,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
14,2499829,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0
15,518072,0.02,0.98,2000,7.59,1.00,0.733,27.0,8.0,11.0,...,5300.0,1941.0,11200.0,80.0,10.0,12.0,20400.0,8130.0,3.0,4.0
16,1205125,0.02,0.98,0,23.67,0.94,0.087,10.0,54.0,53.0,...,2200.0,2200.0,7600.0,73.0,2.0,2.0,16800.0,8900.0,1.0,3.0
18,1129897,0.02,0.98,0,0.25,0.88,0.302,19.0,20.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2599411,0.03,0.65,0,0.31,0.76,0.472,15.0,21.0,14.0,...,5300.0,4750.0,5500.0,79.0,8.0,11.0,19200.0,7987.0,24.0,7.0
26,1413051,0.01,0.99,500,0.80,1.00,0.088,15.0,36.0,35.0,...,2800.0,1520.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,142.0


In [51]:
data1.drop(columns=['latest_query_time','loans_latest_time','trade_no','id_name','bank_card_no','source'],axis=1,inplace=True)

In [55]:
data1['reg_preference_for_trad'].replace({'一线城市':1,'二线城市':2,'三线城市':3,'境外':4,'其他城市':5},inplace = True)

In [56]:
#处理num_feature
num_features_desc = desc(num_features)
num_features_desc

,loss_num,unique_num,unique_percent,valid_num
custid,0,4754,1.000000,4754
low_volume_percent,2,41,0.008418,4752
middle_volume_percent,2,91,0.018939,4752
take_amount_in_later_12_month_highest,0,166,0.034918,4754
trans_amount_increase_rate_lately,3,783,0.164597,4751
trans_activity_month,2,85,0.017677,4752
trans_activity_day,2,513,0.107744,4752
transd_mcc,2,42,0.008628,4752
trans_days_interval_filter,8,148,0.030973,4746
trans_days_interval,2,115,0.023990,4752


In [59]:
num_features.drop(columns=['custid','student_feature'],axis=1,inplace=True)

In [60]:
num_features_desc_new = desc(num_features)
num_features_desc_new

,loss_num,unique_num,unique_percent,valid_num
low_volume_percent,2,41,0.008418,4752
middle_volume_percent,2,91,0.018939,4752
take_amount_in_later_12_month_highest,0,166,0.034918,4754
trans_amount_increase_rate_lately,3,783,0.164597,4751
trans_activity_month,2,85,0.017677,4752
trans_activity_day,2,513,0.107744,4752
transd_mcc,2,42,0.008628,4752
trans_days_interval_filter,8,148,0.030973,4746
trans_days_interval,2,115,0.023990,4752
regional_mobility,2,6,0.001052,4752


In [61]:
#分训练集和测试集
df_1,Y = data1.drop(columns=['status'],axis=1),data1['status']

In [62]:
#利用均值填充缺失值
imputer = Imputer(strategy='median')
df_2 = imputer.fit_transform(df_1)
df_2 = pd.DataFrame(df_2,columns=df_1.columns)

In [64]:
df_2_desc=desc(df_2)
df_2_desc

,loss_num,unique_num,unique_percent,valid_num
custid,0,4754,1.000000,4754
low_volume_percent,0,40,0.008414,4754
middle_volume_percent,0,90,0.018931,4754
take_amount_in_later_12_month_highest,0,166,0.034918,4754
trans_amount_increase_rate_lately,0,782,0.164493,4754
trans_activity_month,0,84,0.017669,4754
trans_activity_day,0,512,0.107699,4754
transd_mcc,0,41,0.008624,4754
trans_days_interval_filter,0,147,0.030921,4754
trans_days_interval,0,114,0.023980,4754


In [67]:
#数据标准化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_3 = scaler.fit_transform(df_2) #生成结果是array类型，需要转化为df类型
df_3 = pd.DataFrame(df_3,columns=df_2.columns)
df_3.head()

,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,1.064536,-0.284251,0.612374,-0.494499,-0.019099,-1.292452,-0.308087,-0.112363,-0.088968,0.258090,...,-0.330249,-0.233181,-1.104758,-0.087098,-1.277708,-0.973381,-1.068511,-1.226123,-0.313460,-0.692110
1,-1.118767,-0.043372,0.267106,0.015242,-0.018551,0.993418,0.544138,0.334643,0.043178,-0.470508,...,0.089632,-0.109253,0.840507,0.267713,0.108626,0.238311,0.490237,0.256592,-0.532163,-0.999858
2,1.120553,0.438384,0.405213,-0.494499,-0.018955,0.993418,-1.477693,-1.006374,1.717020,0.015224,...,-1.239989,-1.008619,-0.684917,0.764448,-1.277708,-1.276304,-0.852018,-0.681007,-0.586839,-0.922921
3,0.114797,-0.525129,0.405213,0.015242,-0.020208,-1.190857,2.419034,1.005152,-0.661598,-0.956239,...,-0.120309,-0.493431,1.008443,0.267713,0.108626,-0.064612,1.009820,0.769001,-0.586839,-0.961389
4,0.782144,-0.284251,0.612374,-0.494499,-0.019733,0.993418,-1.119171,-1.006374,1.628923,1.229553,...,-0.750129,-0.335865,-0.111133,0.196751,-0.931125,-0.973381,-0.548928,0.054899,-0.040081,1.269785


In [70]:
train_x,test_x,train_y,test_y = train_test_split(df_3,Y,train_size=0.7,test_size=0.3,random_state=2018)

In [71]:
print(train_x.shape)
print(train_y.shape)

(3327, 82)
(3327,)


In [72]:
#使用SVM模型进行训练
from sklearn.svm import SVC
clf = SVC()
clf.fit(train_x,train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [73]:
pred_y = clf.predict(test_x)
clf.score(test_x,test_y)

0.7792571829011913

D:\Program Files\anaconda\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
